In [1]:
from utils.data_management import load_data
from models.tf_models import ConvolutionalAE, train

import config as cfg
import warnings

warnings.simplefilter('ignore')

import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import os

print("Number of available GPUs : ", len(tf.config.list_physical_devices('GPU')))

2021-07-27 22:08:50.692416: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


Number of available GPUs :  1


2021-07-27 22:08:51.635173: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2021-07-27 22:08:51.670288: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-07-27 22:08:51.670757: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 2080 with Max-Q Design computeCapability: 7.5
coreClock: 1.095GHz coreCount: 46 deviceMemorySize: 7.79GiB deviceMemoryBandwidth: 357.69GiB/s
2021-07-27 22:08:51.670789: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2021-07-27 22:08:51.673526: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2021-07-27 22:08:51.673557: I tensorflow/s

# Chargement des données

## Chargement des données

In [2]:
# On profite de la puissance de Python : Les dictionnaires
datasets_map = {
    'cifar': None,
    'mnist': None,
    'fashion': None
}

for name in datasets_map.keys():
    datasets_map[name] = load_data(name, cfg.DATA_DIRECTORY)

In [3]:
datasets_map['mnist']['train']['images'].shape

(60000, 1, 28, 28)

# Construction des datasets

In [4]:
cifar_ti = datasets_map['cifar']['train']['images']
cifar_tl = datasets_map['cifar']['train']['labels']
cifar_shape = cifar_ti.shape

cifar_train_dataset = tf.data.Dataset.from_tensor_slices((
    cifar_ti.reshape((cifar_shape[0], cifar_shape[2], cifar_shape[3], cifar_shape[1])),
    cifar_tl
))

2021-07-27 22:08:51.942618: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-07-27 22:08:51.943439: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-07-27 22:08:51.944068: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 2080 with Max-Q Design computeCapability: 7.5
coreClock: 1.095GHz coreCount: 46 deviceMemorySize: 7.79GiB deviceMemoryBandwidth: 357.69GiB/s
2021-07-27 22:08:51.944193: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had ne

In [5]:
cifar_train_dataset

<TensorSliceDataset shapes: ((32, 32, 3), ()), types: (tf.float32, tf.int64)>

# Auto-encodeur

Nous commençons par un auto-encodeur en TensorFlow à l'aide d'une classe personnalisée.

In [6]:
lr = 0.01

model = ConvolutionalAE(latent_size=16)
loss = tf.keras.losses.MeanSquaredError()
optimizer = tf.keras.optimizers.SGD(learning_rate=lr, momentum=0.9)

step_loss = tf.keras.metrics.Mean(name='step_loss')
epoch_loss = tf.keras.metrics.Mean(name='epoch_loss')
loss_operations = (epoch_loss, step_loss)

In [7]:
# Finir de loader le dataset
train_size = len(cifar_train_dataset)
train_dataset = cifar_train_dataset.shuffle(2*train_size).batch(64)
train(model, loss, optimizer, train_dataset, loss_operations, epochs=10)

2021-07-27 22:08:52.742873: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 614400000 exceeds 10% of free system memory.
2021-07-27 22:08:53.362549: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudnn.so.8
2021-07-27 22:08:53.821027: I tensorflow/stream_executor/cuda/cuda_dnn.cc:359] Loaded cuDNN version 8202
2021-07-27 22:08:54.291077: E tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2021-07-27 22:08:54.291105: W tensorflow/stream_executor/gpu/asm_compiler.cc:56] Couldn't invoke ptxas --version
2021-07-27 22:08:54.291518: E tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2021-07-27 22:08:54.291575: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Internal: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This m

Epoch : 1 / 10 - Loss : 0.02995995


2021-07-27 22:09:23.455803: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 614400000 exceeds 10% of free system memory.


Epoch : 2 / 10 - Loss : 0.02550104


2021-07-27 22:09:51.742024: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 614400000 exceeds 10% of free system memory.


Epoch : 3 / 10 - Loss : 0.02257671


2021-07-27 22:10:19.663941: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 614400000 exceeds 10% of free system memory.


Epoch : 4 / 10 - Loss : 0.02047420
Epoch : 5 / 10 - Loss : 0.01890258
Epoch : 6 / 10 - Loss : 0.01765144
Epoch : 7 / 10 - Loss : 0.01664059
Epoch : 8 / 10 - Loss : 0.01580993
Epoch : 9 / 10 - Loss : 0.01509940
Epoch : 10 / 10 - Loss : 0.0144897


In [8]:
# Sauvegarder les poids
n_exp = 1
ae_checkpoint_dir = os.path.join(cfg.TF_EXP_DIRECTORY, "auto_encoder", f"training_{n_exp}")
while os.path.exists(ae_checkpoint_dir):
    n_exp += 1
    ae_checkpoint_dir = os.path.join(cfg.TF_EXP_DIRECTORY, "auto_encoder", f"training_{n_exp}")

ae_checkpoint_path = os.path.join(ae_checkpoint_dir, "ae_model.ckpt")
model.save_weights(ae_checkpoint_path)

### Graph des opération
Je confirme qu'avec **tf.GradientTape()** nous avons bien la construction du graph avec les opérations effectuées durant l'exécution. Le contexte permet d'enregistrer les opération réalisées.

Continuer à se renseigner sur le décorateur **@tf.function** : https://www.tensorflow.org/api_docs/python/tf/function.